In [2]:
sentence = '나는 밥을 먹었다.'

source_sentence = '<start>'+sentence
target_sentence = sentence +'<end>'

print('Source 문장 :', source_sentence)
print('Target 문장 :', target_sentence)

Source 문장 : <start>나는 밥을 먹었다.
Target 문장 : 나는 밥을 먹었다.<end>


<br>**n-1개의 단어 시퀀스(w1, w2, ..., w(n-1))이 주어졌을 때, n번째 단어 w(n)으로 무엇이 올지 예측하는 확률모델을 `언어모델(Language Model`이라고 부른다.**<br><br>

> 어떤 텍스트도 언어모델의 학습데이터가 될 수 있다.<br>
> n-1 번째까지의 단어 시퀀스가 x_train이 되고, n번째 단어가 y_train이 되는 데이터셋은 무궁무진하게 만들 수 있다.<br>

<br>**`GPT-2`와 같은 문장 생성기도 언어 모델의 한 종류에 불과하다.<br>딥러닝 구조나 파라미터 사이즈, 학습데이터의 양 등이 특별할 뿐...**<br><br>

<br>

### 실습 : 연극 대사를 학습해서 스스로 연극 대사 문장을 생성하는 언어모델 인공지능

---
<br>

### (1) 데이터 다듬기

<br>**✅ 데이터 다운로드**<br>

In [1]:
# 실습에 사용할 라이브러리를 불어오자

import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


<br>△ 완벽한 연극대본이지만 화자 이름이나, 공백 데이터는 필요없음 >> 1차 필터링<br><br>
- 화자가 표기된 문장(0, 3, 6) / 공백인 문장(2, 5, 9)

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue  # 길이가 0인 문장은 건너 뜁니다.
    if sentence[-1] == ':': continue # 문장의 끝이 :인 문장은 건너뜁니다.
    
    if idx > 9: break # 일단 문장 10개만 확인해보자
    
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


<br>**`텍스트 생성모델`**에서도(텍스트분류모델처럼) 단어사전을 만들어야 함.<br>문장을 일정한 기준으로 쪼개야 하는데 이를 **`토큰화(Tokenize)`** 라고 함.<br><br>
- 가장 심플한 방법인 띄어쓰기를 기준으로 토큰화 진행해보자<br>

**<토근화 과정 중 발생하는 문제>**
<br>정규 표현식(Regex)를 사용해서 필터링해보자

<img src='https://user-images.githubusercontent.com/103712369/167988613-8eae6d89-b7a9-4c0b-b797-307c33b1d4ff.png' width='700' height='50'/>

<br>**입력된 문장을**<br>
- 1) 소문자로 바꾸고, 양쪽 공백을 지운다.
- 2) 특수문자 양쪽에 공백을 넣고
- 3) 여러개의 공백은 하나의 공백으로 바꾼다
- 4) a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다.
- 5) 다시 양쪽 공백을 지운다.
- 6) 문장 시작에는 <strart>, 끝에는 <end>를 추가한다.<br><br>

In [6]:
ss = '  I am a boy.  .'
print(ss)
print(ss.strip())

  I am a boy.  .
I am a boy.  .


In [7]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', ' ', sentence)  #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>'  #6
    return sentence


# 이 문장이 어떻게 필터링 되는지 확인해보자
print(preprocess_sentence('This @_is ;;; sample     sentence.'))

<br>자연어처리 분야에서 모델의 입력이 되는 문장을 *`소스문장(Source Sentence)`*, <br>정답 역할을 하게 될 모델의 출력 문장을 *`타겟문장(Target Sentence)`*라고 관례적으로 부른다.<br> 각각 X_train, y_train에 해당한다고 볼 수 있다.<br><br>
위에서 만든 정제 함수를 통해 만든 데이터셋에서 토큰화를 진행한 후 끝 단어 `<end>`를 없애면 소스문장,<br>첫 단어 `<start>`를 없애면 타겟 문장이 된다.<br><br>

In [141]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

<br>데이터는 이제 완전하게 준비가 되었다.<br>우리가 가르칠 언어(data)를 인공지능의 모국어인 숫자로 변환해보자<br><br>텐서플로우는 자연어 처리를 위한 여러가지 모듈을 제공하는데 `tf.keras.preprocessing.text.Tokenizer` 패키지는 <U>1) 정제된 데이터를 토큰화</U>하고,<br><U>2) 단어사전(vocabulary 또는 dictionary라고 칭함)을 만들</U>어주며, <U>3) 데이터를 숫자로 변환</U>까지 한방에 해준다.<br><br>이 과정을 **벡터화(vectorize)**라고 하며, 숫자로 변환된 데이터를 **텐서(tensor)**라고 칭한다.<br><br>

In [149]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    #tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[2, 143, 40, 933, 140, 591, 4, 124, 24, 110, 5, 3], [2, 110, 4, 110, 5, 3], [2, 11, 50, 43, 1201, 316, 9, 201, 74, 9, 3034, 15, 3], [2, 1201, 5, 1201, 5, 3], [2, 199, 4, 11, 92, 1021, 298, 18, 2314, 513, 9, 6, 251, 5, 3], [2, 40, 92, 114, 4, 40, 92, 114, 5, 3], [2, 71, 88, 305, 36, 4, 7, 40, 63, 31, 1137, 64, 48, 160, 2063, 5, 3], [2, 18, 114, 13, 4554, 15, 3], [2, 42, 61, 2616, 52, 114, 71, 23, 26, 161, 145, 4, 145, 17, 3], [2, 83, 215, 4, 54, 822, 5, 3], [2, 40, 50, 4555, 171, 822, 4, 6, 1711, 54, 5, 3], [2, 35, 1202, 4556, 52, 67, 3608, 88, 45, 65, 3], [2, 67, 514, 88, 29, 6, 4557, 4, 317, 23, 84, 3], [2, 1712, 4, 40, 239, 861, 65, 6298, 88, 6299, 3], [2, 29, 65, 122, 40, 50, 107, 246, 6, 6300, 14, 3], [2, 6301, 88, 4, 6, 2315, 10, 48, 1582, 4, 18, 32, 94, 3], [2, 6302, 9, 6303, 77, 3609, 48, 3], [2, 2617, 18, 13, 1451, 9, 75, 71, 88, 568, 30, 22, 3], [2, 48, 3610, 4, 237, 40, 592, 6304, 20, 6, 361, 92, 8, 3], [2, 110, 30, 16, 3611, 20, 2316, 4, 19, 16, 3035, 20, 568, 5, 3], [2, 67

In [191]:
tensor[:10]

[[2, 143, 40, 933, 140, 591, 4, 124, 24, 110, 5, 3],
 [2, 110, 4, 110, 5, 3],
 [2, 11, 50, 43, 1201, 316, 9, 201, 74, 9, 3034, 15, 3],
 [2, 1201, 5, 1201, 5, 3],
 [2, 199, 4, 11, 92, 1021, 298, 18, 2314, 513, 9, 6, 251, 5, 3],
 [2, 40, 92, 114, 4, 40, 92, 114, 5, 3],
 [2, 71, 88, 305, 36, 4, 7, 40, 63, 31, 1137, 64, 48, 160, 2063, 5, 3],
 [2, 18, 114, 13, 4554, 15, 3],
 [2, 42, 61, 2616, 52, 114, 71, 23, 26, 161, 145, 4, 145, 17, 3],
 [2, 83, 215, 4, 54, 822, 5, 3]]

In [189]:
len(tensor[:10])

10

In [146]:
print(type(tensor))
print(type(tokenizer))

<class 'numpy.ndarray'>
<class 'keras_preprocessing.text.Tokenizer'>


In [147]:
print(len(corpus))
print(tensor.shape)

24015
(24015, 21)


In [148]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력해 보자

print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


<br>**텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을경우 0으로 패딩(padding)을 채워 넣은 것.<br>
사전에는 없지만 0은 바로 패딩 문자 `<pad>`가 될 것이다.**<br>

In [134]:
# 텐서 데이터는 모두 정수로 이루어짐. 이 숫자는 다름아니라, tokenizer에 구축된 단어 사전의 인덱스

tokenizer.index_word[1]

'<unk>'

<br>텐서 데이터는 모두 정수로 이루어져 있으며 이 숫자는 tokenizer에 구축된 단어사전의 인덱스이다.<br><br>

- `tokenizer.index_word` : 말뭉치(corpus)를 토큰화 시킨 후 단어사전(dictionary 객체)
- `tokenizer.num_words` :  토큰화된 단어의 갯수

In [135]:
tokenizer.num_words

7000

In [136]:
# 단어사전이 어떻게 구축되어 있는지 살펴보면...

for idx in tokenizer.index_word:
    print(idx, ':', tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


<br>이제 생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있도록 한다.<br><br>텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시쿼스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워 넣은 것이다.<br><br>사전에는 없지만 0은 바로 패딩 문자 `<pad>`가 된다.

In [195]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input.shape)
print(tgt_input.shape)

print(src_input[0])
print(tgt_input[0])

TypeError: list indices must be integers or slices, not tuple

In [138]:
print(src_input.shape)
print(len(tgt_input))

(24015, 20)
24015


<br>

- 데이터셋 객체 생성<br><br>

In [77]:
BUFFER_SIZE

24015

In [74]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

#### 이번 스텝에서 데이터셋을 생성하기 위해 거쳐 온 과정을 기억해두자. == `데이터 전처리`

- 정규 표현식을 사용해서 corpus 생성
- `tf.keras.preprocessing.text.Tokenizer`를 이용해 corpus를 텐서로 변환
- `tf.data.Dataset.from_tensor_slices()`를 이용해 corpus 텐서를 `tf.data.Dataset` 객체로 변환

### (2) 인공지능 학습시키기

<img src = 'https://user-images.githubusercontent.com/103712369/168005994-ca087f67-4703-4b3e-b9b0-efc083c326da.png' width ='500' height='50'/>


<br>우리가 만들 모델은 tf.keras.Model을 Subclassing 하는 방식으로 만든다.<br>우리 모델에는 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성됨.

In [79]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [80]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break  # take:: 배치를 불러오는 것.

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 1.09421017e-04, -2.76951378e-05,  1.30810717e-04, ...,
         -1.19838580e-04, -8.85959598e-05,  8.67380295e-05],
        [ 2.63861555e-04,  7.29582280e-06,  4.13964211e-04, ...,
         -2.16087617e-04,  9.29628586e-05,  3.94471048e-04],
        [ 6.18180260e-04, -4.22111734e-05,  6.66090637e-04, ...,
         -2.13982450e-04, -8.73505269e-05,  6.49946101e-04],
        ...,
        [-1.11246819e-03,  9.67179891e-04,  3.35268048e-03, ...,
          1.34330441e-03, -3.22540663e-03, -1.34965917e-03],
        [-1.34266389e-03,  9.28419991e-04,  3.90685629e-03, ...,
          1.45555695e-03, -3.91023466e-03, -1.55078154e-03],
        [-1.56170898e-03,  9.02349944e-04,  4.40225611e-03, ...,
          1.57878129e-03, -4.55145864e-03, -1.73567282e-03]],

       [[ 1.09421017e-04, -2.76951378e-05,  1.30810717e-04, ...,
         -1.19838580e-04, -8.85959598e-05,  8.67380295e-05],
        [ 1.71469626e-04, -5.75500744e-05, -6.

- 모델의 최종 출력 텐서 shape를 유심히 보면 `shape=(256, 20, 7001)`임을 알 수 있다.<br>
- 7001은 Dense 레이어의 출력 차원수.<br>
- 7001개 단어 중 어느 단어의 확률이 가장 높을지를 모델링해야 함.<br><br>

##### 256은 이전 스텝에서 지정한 배치사이즈. `dataset.take(1)`를 통해서 1개의 배치, 즉 256개의 문장 데이터를 가져온다.<br><br>
##### 20은 `tf.keras.layers.LSTM(hidden_size, return_sequences=True)`로 호출한 LSTM 레이어에서 return~=True로 지정한 부분에 있다.<br>즉, LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다.

In [81]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [82]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [83]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [84]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)   ### validation_data 데이터셋을 넣어서 
# validation loss 그래프 그려보자 history 적용해보기

Epoch 1/30
93/93 [==============================] - 37s 376ms/step - loss: 3.4699
Epoch 2/30
93/93 [==============================] - 35s 375ms/step - loss: 2.8043
Epoch 3/30
93/93 [==============================] - 35s 374ms/step - loss: 2.7027
Epoch 4/30
93/93 [==============================] - 35s 376ms/step - loss: 2.6074
Epoch 5/30
93/93 [==============================] - 35s 375ms/step - loss: 2.5423
Epoch 6/30
93/93 [==============================] - 35s 375ms/step - loss: 2.4895
Epoch 7/30
93/93 [==============================] - 35s 376ms/step - loss: 2.4334
Epoch 8/30
93/93 [==============================] - 35s 377ms/step - loss: 2.3821
Epoch 9/30
93/93 [==============================] - 35s 378ms/step - loss: 2.3346
Epoch 10/30
93/93 [==============================] - 35s 377ms/step - loss: 2.2866
Epoch 11/30
93/93 [==============================] - 35s 377ms/step - loss: 2.2383
Epoch 12/30
93/93 [==============================] - 35s 376ms/step - loss: 2.1913
Epoch 13/30
9

### (3) 잘 만들어졌는지 평가하기
**`BLEU` , `ROUGE`**<br><br>

- 이번에는 작문 모델을 평가하기 위해 작문된 문장을 사람이 평가해본다.<br><br>

In [86]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [93]:
generate_text(model, tokenizer, init_sentence="<start> bad")

'<start> bad , sir , to do you courtesy , <end> '